In [1]:
#!pip install findspark

In [11]:
!pip install pyspark --upgrade

In [1]:
if 1:
    #import findspark
    #findspark.init()
    import pyspark
    from pyspark import SparkContext
    sc = SparkContext()
    
    from pyspark.sql import SparkSession
    from pyspark.conf import SparkConf
    spark=SparkSession.builder.config(conf=SparkConf()).getOrCreate()

In [2]:
sc.version

u'2.0.0'

In [3]:
#necessary imports

from sklearn import datasets
from pyspark.sql import SQLContext as SQC
from pyspark.mllib.linalg import Vectors as mllibVs, VectorUDT as mllibVUDT
from pyspark.ml.linalg import Vectors as mlVs, VectorUDT as mlVUDT
from pyspark.sql.types import *
from pyspark.mllib.linalg import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StandardScaler
from pyspark.mllib.linalg.distributed import IndexedRowMatrix
import math as m
import numpy as np

In [4]:
num_samples = 3000
X, color = datasets.make_s_curve(num_samples)


In [5]:
#spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

In [6]:
df = (spark.createDataFrame(sc.parallelize(X.tolist()).zipWithIndex().
                          map(lambda x: (x[1], mlVs.dense(x[0]))), ["id", "features"]))    


In [7]:
# for every point create an id sorted vector of distance from every other point

udf_dist = udf(lambda x, y:  float(x.squared_distance(y)), DoubleType())
df_2 = df


In [8]:
df = df.crossJoin(df).toDF('x_id', 'x_feature', 'y_id', 'y_feature')

AttributeError: 'DataFrame' object has no attribute 'crossJoin'

In [11]:
df = df.withColumn("sim", udf_dist(df.x_feature, df.y_feature))

df = df.drop("x_feature")

st = struct([name for name in ["y_id", "sim","y_feature"]]).alias("map")
df = df.select("x_id", st)
df  = df.groupby("x_id").agg(collect_list("map").alias("map"))
df = df.join(df_2, df_2.id == df.x_id, "inner").drop("x_id")

AttributeError: 'DataFrame' object has no attribute 'crossJoin'